In [1]:
import numpy as np
import pandas as pd

In [7]:
interest_rate = 0.05
recovery = 0.4
hazard_rate = np.array([0.04944286, 0.0667379, 0.0853633, 0.105952, 0.1294527])

In [46]:
class cds():
    def __init__(self, interest_rate, recovery, hazard_rate):
        self.interest_rate = interest_rate
        self.recovery = recovery
        self.hazard_rate = hazard_rate
        self.time = np.linspace(0, len(hazard_rate) , (len(hazard_rate) * 20 + 1))   #time
        self.df = np.exp(-self.time * interest_rate)  #discount factor
        self.running = np.array([300, 350, 400, 450, 500])
        self.up_front = np.zeros(len(hazard_rate))
        
    
    def get_surviv_prob(self):
        # I set the length to be 20 because I'm gonna iterate over the 5 hazard rates, 
        # so for each period the length is 100/5 = 20
        dt = np.ones(20) * 0.05

        # Initializing an array, since at time 0, the csp is 1 and has nothing to do with the hazard rate
        surviv_prob = np.array([1]) 

        # iterate over the hazard rate, and concatenate with the initialized surviv_prob
        for i in range(len(self.hazard_rate)):
            prob = np.exp(- self.hazard_rate[i] * dt) # S(t) = 1 - F(t) = exp(-h * t)
            surviv_prob = np.concatenate([surviv_prob, prob])

        surviv_prob = surviv_prob.cumprod()
        return surviv_prob
    
    
    def get_default_prob(self):
        # F(t) = 1 - exp(-h * t)
        default_prob = 1 - self.get_surviv_prob() 
        return default_prob
    
    def get_marginal_prob(self):
        # marginal default prob equals the older cum survival prob - later prob
        surviv_prob = self.get_surviv_prob()
        marginal_prob = np.concatenate([np.array([0]), surviv_prob[:-1] - surviv_prob[1:]])
        return marginal_prob
    
    def get_defaults(self):
        # marginal default prob equals the older cum survival prob - later prob
        defaults = self.df * self.get_marginal_prob() * (1 - self.recovery)
        return defaults
        
        
    def get_premiums(self):
        # Since the premium is paid by quarter, 
        # so I convert the time in to the index of the quarter by multiplying 4 and then floor down to the nearest integer 
        # then I noticed that the payment happens at the beginning of the quarter num, so I use diff to mark the payment time and
        # this mask array just contains zeros and ones.
        mask = np.floor(self.time * 4)[1:] - np.floor(self.time * 4)[:-1]
        mask = np.concatenate([np.array([0]), mask])

        # Last I just use the formula to calculate the premiums
        premium = mask * self.df * self.get_surviv_prob() * 0.25
        return premium
        
    def get_cds(self):
        # with all the data, I created a dictionary and use it as an input to the dataframe
        data = {'Disc Factor': self.df,
                'Cumulative Survival Probability': self.get_surviv_prob(),
                'Cumulative Default Probability': self.get_default_prob(),
                'Marginal Def Probability': self.get_marginal_prob(),
                'Premiums': self.get_premiums(),
                'Defaults': self.get_defaults()}

        cds = pd.DataFrame(data=data, columns=['Disc Factor','Cumulative Survival Probability','Cumulative Default Probability',
                'Marginal Def Probability', 'Premiums', 'Defaults'], index=self.time)
        return cds
    
    def get_risk_annuity(self):
        # get the risk annuity from premiums column of dataframe 'cds'
        premium = self.get_premiums()
        risk_annuity = premium[np.nonzero(premium)].cumsum()
        idx = 4 * np.array([1, 2, 3, 4, 5]) - 1 # define a index array
        risk_annuity = risk_annuity[idx]
        return risk_annuity
    
    def get_default_leg(self):
        # in the same fashion, we can get the default leg from defaults
        default_leg = self.get_defaults().cumsum()
        idx2 = 20 * (np.arange(5) + 1)
        default_leg = default_leg[idx2]
        return default_leg
    
    def get_mtm(self):
        default_leg = self.get_default_leg()
        risk_annuity = self.get_risk_annuity()
        running = self.running
        up_front = self.up_front
        mtm = default_leg - risk_annuity * running/10000 - up_front
        return mtm

In [65]:
p = cds(interest_rate, recovery, hazard_rate)
p.get_cds().head(11)

,Disc Factor,Cumulative Survival Probability,Cumulative Default Probability,Marginal Def Probability,Premiums,Defaults
0.00,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
0.05,0.997503,0.997531,0.002469,0.002469,0.000000,0.001478
0.10,0.995012,0.995068,0.004932,0.002463,0.000000,0.001470
0.15,0.992528,0.992611,0.007389,0.002457,0.000000,0.001463
0.20,0.990050,0.990160,0.009840,0.002451,0.000000,0.001456
0.25,0.987578,0.987715,0.012285,0.002445,0.243861,0.001449
0.30,0.985112,0.985277,0.014723,0.002439,0.000000,0.001441
0.35,0.982652,0.982844,0.017156,0.002433,0.000000,0.001434
0.40,0.980199,0.980417,0.019583,0.002427,0.000000,0.001427
0.45,0.977751,0.977996,0.022004,0.002421,0.000000,0.001420


In [66]:
x1 = 0.01
x2 = 0.02
# randomly set a test hazard rate
hazard_rate = np.array([ 0.04,  0.06 ,  0.08,  0.10,  0.12])


In [63]:
for i in range(5):
    hazard_rate[i] = x1
    p.hazard_rate = hazard_rate
    mtm = p.get_mtm()
    fl = mtm[i]
    hazard_rate[i] = x2
    p.hazard_rate = hazard_rate
    mtm = p.get_mtm()
    f = mtm[i]
    
    if abs(f1) < abs(f):
        rtsec = x1
        xl = x2
        temp = f
        f = fl
        fl = temp
    else:
        xl = x1
        rtsec = x2
    
    dx = (xl - rtsec) * f / (f - fl)
    epsilon = 0.000001
    while (abs(dx) > epsilon and f != 0):
        dx = (xl - rtsec) * f / (f - fl) 
        xl = rtsec
        fl = f
        rtsec = rtsec + dx
        hazard_rate[i] = rtsec
        p.hazard_rate = hazard_rate
        mtm = p.get_mtm()
        f = mtm[i]
        print 'iteration ', str(i)
        print hazard_rate
        print 'xl =', str(xl)


iteration  0
[ 0.04887781  0.06        0.08        0.1         0.12      ]
xl = 0.02
iteration  0
[ 0.04943472  0.06        0.08        0.1         0.12      ]
xl = 0.048877814196
iteration  0
[ 0.04944286  0.06        0.08        0.1         0.12      ]
xl = 0.0494347152745
iteration  0
[ 0.04944286  0.06        0.08        0.1         0.12      ]
xl = 0.0494428583175
iteration  1
[ 0.04944286  0.06545648  0.08        0.1         0.12      ]
xl = 0.01
iteration  1
[ 0.04944286  0.06670252  0.08        0.1         0.12      ]
xl = 0.0654564823378
iteration  1
[ 0.04944286  0.06673788  0.08        0.1         0.12      ]
xl = 0.0667025164278
iteration  1
[ 0.04944286  0.0667379   0.08        0.1         0.12      ]
xl = 0.066737881339
iteration  2
[ 0.04944286  0.0667379   0.0830002   0.1         0.12      ]
xl = 0.01
iteration  2
[ 0.04944286  0.0667379   0.08527702  0.1         0.12      ]
xl = 0.0830002009559
iteration  2
[ 0.04944286  0.0667379   0.08536321  0.1         0.12      ]
